# Lattice View for ColList

This notebook computes `ColList` using Range–CoMine on a small dataset and **visualizes the pattern lattice** grouped by critical distance levels.

In [ ]:

# Imports
from range_comine.synthetic import generate_synthetic
from range_comine.mining import range_comine
import itertools, math
import matplotlib.pyplot as plt


In [ ]:

# Generate data and run
objs = generate_synthetic(n_features=4, instances_per_feat=5, seed=7)
ColList = range_comine(objs, d1=8.0, d2=30.0, min_prev=0.5)
ColList


## Helper: subset relation
We’ll treat each pattern as a sorted tuple of feature labels and connect nodes if one is a **(k−1)**-subset of the other.

In [ ]:

def as_tuple(p):
    return tuple(sorted(p))

# Build a level-wise structure: list of (d, [patterns])
levels = sorted([(d, [as_tuple(p) for p in pats]) for d, pats in ColList.items()], key=lambda x: x[0])

# Map pattern -> (level_idx, x_pos)
positions = {}
edges = []  # (parent, child)
x_spacing = 1.6

for li, (d, pats) in enumerate(levels):
    # group by size for nicer horizontal spread
    by_k = {}
    for p in pats:
        by_k.setdefault(len(p), []).append(p)
    xs = {}
    # lay out by_k in ascending k
    x_cursor = 0.0
    for k in sorted(by_k.keys()):
        group = sorted(set(by_k[k]))
        for p in group:
            positions[p] = (li, x_cursor)
            x_cursor += x_spacing
    # edges: if q is a superset of p and |q|=|p|+1 and both present (maybe at the same level if same critical distance)
for (li, (d, pats)) in enumerate(levels):
    set_pats = set(pats)
    for q in pats:
        for i in range(len(q)):
            parent = tuple(q[:i] + q[i+1:])
            if parent in set_pats:
                edges.append((parent, q))


## Plot lattice
Nodes are grouped along the vertical axis by **critical distance** level; arrows show subset → superset (if they share the same critical distance).

In [ ]:

plt.figure(figsize=(10, 6))

# y = level index (0 at bottom); we invert to have smallest critical distance at bottom
for p, (li, x) in positions.items():
    y = li
    label = "".join(p)
    plt.scatter([x], [y])
    plt.text(x, y + 0.05, label, ha="center", va="bottom", fontsize=10)

# draw edges (subset to superset) within the same level
for a, b in edges:
    ya = positions[a][0]
    yb = positions[b][0]
    if ya == yb:
        xa = positions[a][1]
        xb = positions[b][1]
        plt.arrow(xa, ya, xb - xa, yb - ya, length_includes_head=True, head_width=0.08, head_length=0.12)

# y ticks as critical distance values
y_ticks = list(range(len(levels)))
y_labels = [f"d={d:.2f}" for (d, _) in levels]
plt.yticks(y_ticks, y_labels)

plt.xlabel("Patterns laid out horizontally")
plt.ylabel("Critical distance levels")
plt.title("ColList Lattice (within-level subset links)")
plt.grid(True, linestyle="--", alpha=0.4)
plt.tight_layout()


### Tip
If you want cross-level edges (parent at smaller critical distance than child), uncomment the following cell and re-run.

In [ ]:

# Cross-level edges (optional)
# plt.figure(figsize=(10, 6))
# for p, (li, x) in positions.items():
#     y = li
#     label = "".join(p)
#     plt.scatter([x], [y])
#     plt.text(x, y + 0.05, label, ha="center", va="bottom", fontsize=10)
# for li, (d, pats) in enumerate(levels):
#     for q in pats:
#         for i in range(len(q)):
#             parent = tuple(q[:i] + q[i+1:])
#             # search parent at any level
#             if parent in positions:
#                 xa, ya = positions[parent]
#                 xb, yb = positions[q]
#                 plt.arrow(xa, ya, xb - xa, yb - ya, length_includes_head=True, head_width=0.08, head_length=0.12)
# y_ticks = list(range(len(levels)))
# y_labels = [f"d={d:.2f}" for (d, _) in levels]
# plt.yticks(y_ticks, y_labels)
# plt.xlabel("Patterns laid out horizontally")
# plt.ylabel("Critical distance levels")
# plt.title("ColList Lattice (cross-level subset links)")
# plt.grid(True, linestyle="--", alpha=0.4)
# plt.tight_layout()
